**important note** until this [issue](https://github.com/modelcontextprotocol/python-sdk/issues/156) is fixed stderr is not logged in jupyter environment this means that if your `mcp_server_parameters` have an error or the subprocess running the mcp server fail you won't know about it appart from the fact that the cell just hangs. For now the recommendation would be to try it in a regular python script or to temporarly change `mcp/client/stdio.py`:

```python
process = await anyio.open_process(
        [server.command, *server.args],
        env=server.env if server.env is not None else get_default_environment(),
        stderr=None,
 )
```

### Async Usage

Async should just work and is the preferred approach if your agentic framework supports async.



In [1]:
import os
from typing import Any, Callable, Coroutine

from mcpadapt.core import MCPAdapt, ToolAdapter
import mcp

In [2]:
# define a dummy tool adapter that just returns the function for the example.
class DummyToolAdapter(ToolAdapter):
    def adapt(
        self,
        func: Callable[[dict | None], mcp.types.CallToolResult],
        mcp_tool: mcp.types.Tool,
    ):
        return func

    def async_adapt(
        self,
        afunc: Callable[[dict | None], Coroutine[Any, Any, mcp.types.CallToolResult]],
        mcp_tool: mcp.types.Tool,
    ):
        return afunc

In [3]:
# define the mcp server parameters to run
serverparams = mcp.StdioServerParameters(
    command="uv",
    args=["--quiet", "run", "../src/echo.py"],
    env={"UV_PYTHON": "3.12", **os.environ},
)

In [4]:
async with MCPAdapt(serverparams, adapter=DummyToolAdapter()) as tools:
    echo_tool = tools[0]
    response = await echo_tool({"text": "hello"})
    print(response)

meta=None content=[TextContent(type='text', text='hello')] isError=False


### Sync Usage

Similary sync should just work as well with the same issues about stderr as above until resolution of the issue.

In [5]:
with MCPAdapt(serverparams, adapter=DummyToolAdapter()) as tools:
    echo_tool = tools[0]
    response = echo_tool({"text": "hello"})
    print(response)

meta=None content=[TextContent(type='text', text='hello')] isError=False
